## Improving Amazon Transcribe transcriptions using Custom Vocabularies, Amazon Augmented AI (A2I), and Amazon SageMaker.



Visit https://github.com/aws-samples/amazon-a2i-sample-jupyter-notebooks for all A2I Sample Notebooks
